In [2]:
from tabula import read_pdf

In [126]:
import re
from tika import parser
import pandas as pd

In [179]:
def extract_pdf(filename, start_col=0):
    raw = parser.from_file(filename)
    data = raw['content']
    data = re.sub(r'\n+(<.+>?)\n', '', data)
    data = re.sub(r'(\n.+2562\n?)', '', data)
    data = re.sub(r'(จังหวัด.+พรรค)\n', '', data)
    data = re.sub(r'\n\n\n', '\n', data).strip()
    data = list(map(lambda r: r.split(' ')[start_col:], filter(lambda x: x, data.split('\n'))))
    data = list(map(lambda r: r[:-1] if r[-1] == '' else r, data))
    return data

In [180]:
rows = extract_pdf("./raw-files/article_20190205155606.pdf", start_col=1) +  \
    extract_pdf("./raw-files/article_20190206164217.pdf", start_col=0)

# rows = list(map(lambda r: r.split(' '), filter(lambda x: x, rows.split('\n'))))

In [181]:
good_rows = filter(lambda r: len(r) == 7, rows)
bad_rows = list(filter(lambda r: 1 < len(r) < 7 or len(r) > 7, rows))

In [182]:
df = pd.DataFrame(good_rows, columns=['provice', 'zone', 'title', 'name', 'surname', 'no.', 'party'])

In [183]:
len(df) + len(list(bad_rows))

6836

In [191]:
for i, r in enumerate(bad_rows):
    if len(r) > 7:
        print(' '.join(r))
    if i % 5 == 0:
        print('-------')

1 กระบี่ 1 นาย วรวงค์ ระฆังทอง 1 ประชาภิวัฒน์
-------
กระบี่ 1 ว่าท่ี พ.ต. กิตติศักดิ์ กิตติสิทโธ 10 ภูมิใจไทย
กระบี่ 2 นาย อัศวโรจน์ เถาว์กลอย กูจิ 2 เพื่อไทย
กรุงเทพมหานคร 14 ว่าท่ี ร.ท. วีรพล วงษ์มะเซาะ 19 รักษ์ผืนป่าประเทศไทย
กรุงเทพมหานคร 15 ว่าท่ี ร.ต. ศิริชัย เอมสอาด 5 ประชาภิวัฒน์
กรุงเทพมหานคร 16 นาย เทวา ปาลกะวงศ์ ณ อยุธยา 3 ชาติไทยพัฒนา
-------
กรุงเทพมหานคร 16 นาง ฮูวัยดีย๊ะ พิศสุวรรณ อุเซ็ง 12 ประชาธิปัตย์
กรุงเทพมหานคร 22 พันตํารวจโท วัลลภ ณ นคร 6 ประชาชาติ
กรุงเทพมหานคร 22 นาย จุมพฏ กระบวนยุทธ เลขะพันธ์ฎ 17 รักษ์ผืนป่าประเทศไทย
กรุงเทพมหานคร 26 ว่าท่ี ร.ต. ธนกร เจียโหงว 5 เสรีรวมไทย
กรุงเทพมหานคร 29 น.อ. ประพันธ์ โชติอาภรณ์ ร.น. 10 พลังธรรมใหม่
-------
กรุงเทพมหานคร 30 ว่าท่ี ร.ต. ณัฐฐากูร ศรีธนกฤตชโยดม 13 ประชาชาติ
กําแพงเพชร 3 นาง พจณิชา  นาก้อนทอง 8 พลังท้องถ่ินไท
ขอนแก่น 5 ว่าท่ี ร.ต. โกวิทย์ บุตรลําภู 11 ชาติพัฒนา
ชลบุรี 2               ร.ท. ธนัท อโสโก 11 รักษ์ผืนป่าประเทศไทย
ชัยนาท 2 นาย จิระ  เมธีวิวัฒน์ 2 เพื่อชาติ
-------
ชุมพร 2 พ.อ. พรพจน์ โชติ ณ ภาลัย 16 เพื่

In [188]:
df.to_csv('election-62-candidates-merged-before-fix.csv')